<a href="https://colab.research.google.com/github/michael-L-i/DWave/blob/master/Factoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 1. Solving the factoring problem algebraically

The factoring problem: Given a  large number $M$, the gaol is find the two prime factors $p$ and $q$ such that
$$
M = p q.
$$
For instance, if $M=15$ then $p=3$ and $q=5$. To solve this problem on D-Wave (using quantum annealing) we have to map the problem of finding $p$ and $q$ from the integer domain to the binary domain. Let's say, we would like to factor $M=15$. First, we write 
$$
 p = p_0 + 2 p_1 + 4 p_2\, \mbox{and}\, q = q_0 + 2 q_1 + 4 q_2. 
$$
Now the prolem becomes: find the $p_i$ and $q_i$ such that
$$
M = (p_0 + 2 p_1 + 4 p_2) (q_0 + 2 q_1 + 4 q_2).
$$
Let's do the calculations:


In [ ]:
from sympy import *
p = IndexedBase("p")
q = IndexedBase("q")

In [ ]:
eq = 15 - ((p[0] + 2* p[1] + 4 *p[2]) * (q[0] + 2 *q[1] + 4 *q[2]) )
expand(eq)

-p[0]*q[0] - 2*p[0]*q[1] - 4*p[0]*q[2] - 2*p[1]*q[0] - 4*p[1]*q[1] - 8*p[1]*q[2] - 4*p[2]*q[0] - 8*p[2]*q[1] - 16*p[2]*q[2] + 15

The goal is now to solve $eq =0$. As the title of this section indicates, it is possible to solve this algebraically {\it without a quantum computer} (but not efficiently). Let's explain how this before we move ahead with quantum annealing: Let's state the problem one more time:

Find $p_i$s and $q_i$s such that:

*   $eq=0$ where $eq$ is given in the above cell
*   all $p_i$s and $q_i$s are binary: 
$p_i (p_i-1)=0$ and $q_i (q_i-1)=0$. So we ended up with a set of algebraic equations
$$
  \mathcal S = \{eq=0, p_0 (p_0-1)=0,p_1 (p_1-1)=0,p_2 (p_2-1)=0, q_0 (q_0-1)=0, q_1 (q_1-1)=0, q_2 (q_2-1)=0 \}.
$$
This system can be constructed as follows: 




In [ ]:
S = [expand(eq)]
for i in range(3):
  S += [p[i]*(p[i]-1), q[i]*(q[i]-1)]
S

[-p[0]*q[0] - 2*p[0]*q[1] - 4*p[0]*q[2] - 2*p[1]*q[0] - 4*p[1]*q[1] - 8*p[1]*q[2] - 4*p[2]*q[0] - 8*p[2]*q[1] - 16*p[2]*q[2] + 15,
 (p[0] - 1)*p[0],
 (q[0] - 1)*q[0],
 (p[1] - 1)*p[1],
 (q[1] - 1)*q[1],
 (p[2] - 1)*p[2],
 (q[2] - 1)*q[2]]

To solve the system $\mathcal S$ we use Groebner bases technique:

In [ ]:
groebner(S, p[0], p[1], p[2], q[0], q[1], q[2] , order='lex')

GroebnerBasis([p[0] - 1, p[1] - q[2], p[2] + q[2] - 1, q[0] - 1, q[1] + q[2] - 1, q[2]**2 - q[2]], p[0], p[1], p[2], q[0], q[1], q[2], domain='ZZ', order='lex')

In [ ]:
solve([p[0] - 1, p[1] - q[2], p[2] + q[2] - 1, q[0] - 1, q[1] + q[2] - 1, q[2]**2 - q[2]], p[0], p[1], p[2], q[0], q[1], q[2] )

[{p[0]: 1, p[1]: 0, p[2]: 1, q[0]: 1, q[1]: 1, q[2]: 0},
 {p[0]: 1, p[1]: 1, p[2]: 0, q[0]: 1, q[1]: 0, q[2]: 1}]

Next we generalize this procedure: 

In [ ]:
from math import log2 
from math import *

Notice that $M=pq$ implies that $log_2(M) = log_2(p) + log_2(q)$, which means the number of bits allocated to $p$ and $q$ adds up to the number of bit in $M$ (the number of bits being $ceiling(log_2(x))$). This will guarantee we are not injecting more variables than we are suposed to. 

In [ ]:
def factorBiPrime(M):
  ceiling = ceil

  p_eq =0 
  q_eq = 0
  varlist = []
  S = []

  Mb = ceiling(log2(M))
  Pb = ceiling(log2(M)/2)+1
  Qb = Mb-Pb +1 

  for i in range (Pb):
    p_eq += p[i]*2**i
    varlist += [p[i]]
    S += [p[i]*(p[i]-1)]

  for i in range(Qb):
    q_eq += q[i]*2**i 
    varlist += [q[i]] 
    S += [q[i]*(q[i]-1)]

  eq = M - (p_eq)*(q_eq)
  S += [eq]
    

  sol = (solve(groebner(S, varlist , order='lex'), varlist))
  
  for x in sol.items():
    p_eq = p_eq.subs(x[0], x[1])
    q_eq = q_eq.subs(x[0], x[1])
  return p_eq, q_eq

In [ ]:
factorBiPrime(77)

(11, 7)

# 2. Solving the factoring problem using DWAVE (first approach)

The goal is to solve $eq=0$ using QA. However is DWAVE is a minimizer i.e., finds the minimum of the cost function. Therefore, we have to map our problem of finding the zeros (solutions) of the equation $eq=0$ into finding the minimum of a certain function. We do this with the following observation:

The solutions ($pi$ and $qi$) of the equation eq =0 are exactly the minima of the cost function $Q = eq^2$. Recall 
$$
eq = 15 - ((p_0 + 2 p_1) (q_0 + 2 q_1 + 4q_2) ).
$$

In [ ]:
eq = 15 - ((p[0] + 2* p[1] + 4 *p[2]) * (q[0] + 2 *q[1] + 4 *q[2]) )

In [ ]:
preQ = expand(eq**2)
preQ

p[0]**2*q[0]**2 + 4*p[0]**2*q[0]*q[1] + 8*p[0]**2*q[0]*q[2] + 4*p[0]**2*q[1]**2 + 16*p[0]**2*q[1]*q[2] + 16*p[0]**2*q[2]**2 + 4*p[0]*p[1]*q[0]**2 + 16*p[0]*p[1]*q[0]*q[1] + 32*p[0]*p[1]*q[0]*q[2] + 16*p[0]*p[1]*q[1]**2 + 64*p[0]*p[1]*q[1]*q[2] + 64*p[0]*p[1]*q[2]**2 + 8*p[0]*p[2]*q[0]**2 + 32*p[0]*p[2]*q[0]*q[1] + 64*p[0]*p[2]*q[0]*q[2] + 32*p[0]*p[2]*q[1]**2 + 128*p[0]*p[2]*q[1]*q[2] + 128*p[0]*p[2]*q[2]**2 - 30*p[0]*q[0] - 60*p[0]*q[1] - 120*p[0]*q[2] + 4*p[1]**2*q[0]**2 + 16*p[1]**2*q[0]*q[1] + 32*p[1]**2*q[0]*q[2] + 16*p[1]**2*q[1]**2 + 64*p[1]**2*q[1]*q[2] + 64*p[1]**2*q[2]**2 + 16*p[1]*p[2]*q[0]**2 + 64*p[1]*p[2]*q[0]*q[1] + 128*p[1]*p[2]*q[0]*q[2] + 64*p[1]*p[2]*q[1]**2 + 256*p[1]*p[2]*q[1]*q[2] + 256*p[1]*p[2]*q[2]**2 - 60*p[1]*q[0] - 120*p[1]*q[1] - 240*p[1]*q[2] + 16*p[2]**2*q[0]**2 + 64*p[2]**2*q[0]*q[1] + 128*p[2]**2*q[0]*q[2] + 64*p[2]**2*q[1]**2 + 256*p[2]**2*q[1]*q[2] + 256*p[2]**2*q[2]**2 - 120*p[2]*q[0] - 240*p[2]*q[1] - 480*p[2]*q[2] + 225

We are not done yet because we still have to map $preQ$ to a quadratic cost function $Q$. DWAVE takes only quadratic cost functions (Ising models). 